In [5]:
import threading
from pathlib import Path

import lib_ipmu_drv_config as config_drv

import lib_ipmu_daq_api as api
import lib_ipmu_daq_config as config_daq
import lib_ipmu_daq_generator as gen
#import lib_ipmu_daq_aquisition as daq
import lib_ipmu_daq_process as post
import lib_ipmu_daq_graph as graph

import lib_ipmu_drv_config as config_drv
# import lib_ipmu_drv_command as command

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
preset = config_daq._loadConfig(Path("./_config_preset.toml"))
run = config_drv._loadConfig(Path("./_config_run.toml"))
app_config = config_daq.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.139000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 36000
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 5

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.020000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  device_com          : COM5
  target_speed_rps    : 1.000000
  current_init        : 45
  electrical_angle_init: 200
  t_DC                : 90
  pps                 : 100
  step                : 20
  rst                 : 1
  ppsps               : 2000
  t_stablerot         : 7200
  t_excess_spindown   : 3600
  dir_rotatio

In [7]:
drv_config = config_drv.initParams(run, debug=True)


[Driver]
  device_com          : COM5
  target_speed_rps    : 1.000000
  current_init        : 45
  electrical_angle_init: 200
  t_DC                : 90
  pps                 : 100
  step                : 20
  rst                 : 1
  ppsps               : 2000
  t_stablerot         : 7200
  t_excess_spindown   : 3600
  dir_rotation        : 1
  t_current_reduction_duration: 300
  step_current_reduction: [45, 40, 35, 30, 25, 20, 15, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

[Dependent]
  target_speed_rpm    : 60
  electrical_frequency: 36.000000
  pps_fin             : 35784
  t_total             : 12674.200000


In [8]:
try:
    app = api.DAQApp(config=app_config, DEBUG=True)
    app.setup() # initialize widgets
    app.initStorer(runs_dir=config_daq.RUNS_DIR) # initialize HDF5 storage
    app.initLogger()  # Initialize logger in the main thread to avoid issues


    # generater for debug
    generator = gen.Generator(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)
    
    # Motor
    # motor = command.Command(config=drv_config, comvel_q=app.comvel_q, stop_event=app.stop_event)
    # motor._reset()
    # motor.setup(DEBUG=True)

    # acquisiton for measurements
    # acquisition = daq.DataAquisition(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)

    # processor
    #processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, comvel_q=app.comvel_q, DataStoreFlag = motor.DataStoreFlag, stop_event=app.stop_event, h5f=app.h5f, dset=app.dset, debug=app.DEBUG, logger=app.logger)

    processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, comvel_q=app.comvel_q, DataStoreFlag = None, stop_event=app.stop_event, h5f=app.h5f, dset=app.dset, debug=app.DEBUG, logger=app.logger)

    # Graph
    gui = graph.DAQGUI(config=app_config, quad_q=app.quad_q, stop_event=app.stop_event)

    # 5. Create the thread objects individually
    #logger_thread = threading.Thread(target=app.initLogger, daemon=True)
    generator_thread = threading.Thread(target=generator.run, daemon=True)
    #acquisition_thread = threading.Thread(target=acquisition.run, daemon=True)
    processor_thread = threading.Thread(target=processor.run, daemon=True)
    # motor_thread = threading.Thread(target=motor.run, daemon=True)

    # 6. Start the threads (now controlled from the top level)
    # logger_thread.start()
    generator_thread.start()
    # acquisition_thread.start()
    processor_thread.start()
    # motor_thread.start()
    
    gui.run()

finally:
    app.shutdown()
    #logger_thread.join(timeout=0.1)
    generator_thread.join(timeout=0.1)
    
    # acquisition_thread.join(timeout=0.1)
    processor_thread.join(timeout=0.1)


DEBUG mode is ON.
HDF5 dataset created at: C:\Users\USER\Desktop\ipmu_DAQ\src\..\runs\251118144800.h5
EPOCH = 280725.238802, wall = 230.98 ms, jitter = 230.98 ms  delta c=+5, v= 0.018, v_ref= 0.000, time_p = 0.069495, P_tot_sum = 0.415731, P_tot_w = 0.836397
EPOCH = 280725.367436, wall = 128.63 ms, jitter = 128.63 ms  delta c=+6, v= 0.021, v_ref= 0.000, time_p = 0.208495, P_tot_sum = 0.415694, P_tot_w = 0.555623
EPOCH = 280726.031901, wall = 664.47 ms, jitter = 664.47 ms  delta c=+6, v= 0.021, v_ref= 0.000, time_p = 0.347495, P_tot_sum = 0.415738, P_tot_w = 0.081157
EPOCH = 280726.150093, wall = 118.19 ms, jitter = 118.19 ms  delta c=+5, v= 0.018, v_ref= 0.000, time_p = 0.486495, P_tot_sum = 0.415750, P_tot_w = 0.069624
EPOCH = 280726.373606, wall = 223.51 ms, jitter = 223.51 ms  delta c=+6, v= 0.021, v_ref= 0.000, time_p = 0.625495, P_tot_sum = 0.415712, P_tot_w = 0.537036
EPOCH = 280726.502897, wall = 129.29 ms, jitter = 129.29 ms  delta c=+6, v= 0.021, v_ref= 0.000, time_p = 0.76449